In [ ]:
import pandas as pd
import folium 
import seaborn
import os
import json
# Used for downloading and handling files:
import gzip
import urllib
import pycountry
#import requests

In [ ]:
folium.__version__

# 01 - Unemployment in European countries



### With xls files

In [ ]:
#urllib.request.urlretrieve('http://ec.europa.eu/eurostat/tgm/web/_download/Eurostat_Table_tsdec450NoFlagNoDesc_f375eece-81ae-4fc9-b4ef-5ab165fe8c72.xls',
                          #'eurostat.xls')
na_values= [':', '(i)', '(p)']
df = pd.read_excel('eurostat.xls', skiprows=range(3), skip_footer=9, 
                          na_values=na_values, header=0)
df = df.rename(columns={'geo\\time': 'country'})
df = df.set_index('country')
df = df.iloc[4:, 0:len(df.columns):2]
df.head()

In [ ]:
m_eu = folium.Map([51, 15], zoom_start=4)
eu_topo_path = r'topojson/europe.topojson.json'
folium.TopoJson(open(eu_topo_path),
                'objects.europe',
                name='topojson',
               ).add_to(m_eu)
m_eu

In [ ]:
# todo: find a way to drop switzerland value in json file
eu_json = json.load(open(eu_topo_path))

In [ ]:
df[df['2016'].isnull()]

In [ ]:
eu_json['objects']['europe']

In [ ]:
q = list(df['2016'].quantile([.05, .235, .655, .774, .9455]))
m_europe = folium.Map(location=[46, 15], tiles='cartodbpositron', zoom_start =4)
m_europe.choropleth(geo_data=eu_json, data=df.loc[:,'2016'].dropna(),
                columns = ['country', '2016'],
                threshold_scale = q,
                key_on='feature.properties.NAME',
                topojson='objects.europe',
                fill_color='YlGn',
                fill_opacity=0.7,
                line_opacity=0.2,
                legend_name='Unemployment Rate (%)')

m_europe

### With tsv files
Still need to change country code into strings

In [ ]:
# Downloading and reading in the data
#urllib.request.urlretrieve(
       # 'http://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/tipsun20.tsv.gz', 
        #'tipsun20.tsv.gz')
file = gzip.open('tipsun20.tsv.gz')
df = pd.read_csv(file, sep='\t')

# Re-organising the columns

df = df.rename(columns={'sex,age,unit,geo\\time': 'country'})
#df = df.set_index('country')
df = df[ df.country.str.contains('TOTAL') ] # filter out extra data, only interested in total unemployment rate
df.head()

In [ ]:
pycountry.countries.get(alpha_2='GB').name

In [ ]:
# WIP
for i in range(len(df.country)):
    try:
        df.loc[i, 'country'] = pycountry.countries.get(alpha_2=df.loc[i, 'country'][15:]).name 
    except:
        print('No country code for:', df.loc[i, 'country'][15:])
    
    #print(df.loc[i, 'country'])

ISO-3166-1993 country codes for Estonia and United Kingdom are: `EE` and `GB`

In [ ]:
df[df.loc[:, 'country'].str.contains('EL')].country = 'EL'# = pycountry.countries.get(alpha_2='EE').name 
#df[df.loc[:, 'country'].str.contains('UK'), 'country']# = pycountry.countries.get(alpha_2='GB').name 
df

# 02 - Unemployment in Swiss cantons

In [ ]:
canton_json = json.load(open(canton_topo_path))

In [ ]:
canton_json['objects']['cantons']

In [ ]:
m_ch = folium.Map([47,8.3], zoom_start=8)
canton_topo_path = r'topojson/ch-cantons.topojson.json'
#m_ch.geo_json(geo_path=canton_topo_path, topojson='objects.cantons')
#m_ch.geo_json

folium.TopoJson(open(canton_topo_path),
                'objects.cantons',
                name='topojson',
               ).add_to(m_ch)
m_ch

In [ ]:
df.head()